# 3 - Node distance

Notebook exploring the euclidean distance calculation.
The nearest neighbor edge builder is tested here.

In [3]:
import numpy as np
import awkward as ak
import uproot

In [4]:
filename = "ntuples_13723003_0.root"
file = uproot.open(filename)

In [5]:
tracksters = file["ticlNtuplizer/tracksters"]

In [7]:
vertices_indexes = tracksters['vertices_indexes'].array()

In [8]:
events = tracksters['event'].array()

In [9]:
energy = tracksters['raw_energy'].array()

In [10]:
vertices_x = tracksters['vertices_x'].array()
vertices_y = tracksters['vertices_y'].array()
vertices_z = tracksters['vertices_z'].array()

In [26]:
ver_x = ak.to_numpy(vertices_x[0,0])
ver_y = ak.to_numpy(vertices_y[0,0])
ver_z = ak.to_numpy(vertices_z[0,0])
diff_x = np.subtract.outer(ver_x,ver_x)
diff_y = np.subtract.outer(ver_y,ver_y)
diff_z = np.subtract.outer(ver_z,ver_z)

In [28]:
euclidean_matrix = np.sqrt(diff_x**2+diff_y**2+diff_z**2)

In [32]:
ver_E = ak.to_numpy(tracksters['vertices_energy'].array()[0,0])

In [44]:
max_val = np.max(euclidean_matrix[0][ver_E > ver_E[0]])

In [42]:
euclidean_matrix[0] == min_val

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [54]:
vertices_indexes00 = vertices_indexes[0,0]

In [57]:
for i in range(len(vertices_indexes00)):
    dist_array = euclidean_matrix[i][ver_E > ver_E[i]]
    if len(dist_array) == 0:
        continue
    min_val = np.min(dist_array)
    idx = np.where(euclidean_matrix[i] == min_val)[0][0]
    print("Node %d should be connected with node %d"%(
        vertices_indexes00[i],
        vertices_indexes00[idx])
    )

Node 1 should be connected with node 9
Node 9 should be connected with node 26
Node 26 should be connected with node 33
Node 33 should be connected with node 45
Node 43 should be connected with node 45
Node 59 should be connected with node 45
Node 63 should be connected with node 59
Node 72 should be connected with node 88
Node 88 should be connected with node 113
Node 98 should be connected with node 113
Node 109 should be connected with node 112
Node 112 should be connected with node 135
Node 113 should be connected with node 59
Node 134 should be connected with node 182
Node 135 should be connected with node 113
Node 136 should be connected with node 135
Node 159 should be connected with node 161
Node 160 should be connected with node 159
Node 161 should be connected with node 183
Node 181 should be connected with node 182
Node 182 should be connected with node 184
Node 183 should be connected with node 135
Node 184 should be connected with node 135
Node 204 should be connected with

In [65]:
from src.functions import euclideanMatrix

In [83]:
vertices_E = tracksters['vertices_energy'].array()
edges = ak.ArrayBuilder()
for ide, v_event in enumerate(vertices_indexes):
    edges.begin_list()
    for idt, v_trackster in enumerate(v_event):
        euMatr = euclideanMatrix(
            vertices_x[ide,idt],
            vertices_y[ide,idt],
            vertices_z[ide,idt]
        )
        edges.begin_list()
        for idn, v_node in enumerate(v_trackster):
            higherE = vertices_E[ide,idt] > vertices_E[ide,idt,idn]
            dist_array = euMatr[idn][higherE]
            if len(dist_array) == 0:
                continue
            min_val = np.min(dist_array)
            idx = np.where(euMatr[idn] == min_val)[0][0]
            edges.begin_tuple(2)
            edges.index(0).integer(v_node)
            edges.index(1).integer(v_trackster[idx])
            edges.end_tuple()
        edges.end_list()
    edges.end_list()

In [84]:
edges

<ArrayBuilder [[[(1, 9), (9, 26, ... ), (941, 873)]]] type='100 * var * var * (i...'>

In [86]:
edges[0,0]

<Array [(1, 9), (9, 26), ... 204), (261, 257)] type='26 * (int64, int64)'>